In [1]:
import pandas as pd
from time import time
import matplotlib.pyplot as plt
import matplotlib
from sklearn import preprocessing
import pickle

%matplotlib inline
matplotlib.rcParams['font.sans-serif'] = ['SimHei']

In [2]:
def load_obj(name ):
    with open('' + name + '.pkl', 'rb') as f:
        return pickle.load(f)


In [26]:
BenignDistributeObj = load_obj("Benign_Distribution_set")
print(BenignDistributeObj["2"])
DDoSDistributeObj = load_obj("DDoS_Distribution_set")
print(DDoSDistributeObj["2"])
PortScanDistributeObj = load_obj("PortScan_Distribution_set")
print(PortScanDistributeObj["2"])
Examples = load_obj("Examples")

{'0-5': 97341, '5-10': 242, '10-15': 57, '15-20': 28, '20-25': 10, '25-30': 16, '30-35': 7, '35-40': 3, '40-45': 0, '45-50': 1, '50-55': 1, '55-60': 2, '60-65': 2, '65-70': 1, '70-75': 2, '75-80': 0, '80-85': 2, '85-90': 1, '90-95': 1, '95-100': 1}
{'0-5': 47, '5-10': 1453, '10-15': 1453, '15-20': 0, '20-25': 56728, '25-30': 0, '30-35': 23180, '35-40': 22301, '40-45': 22301, '45-50': 0, '50-55': 1189, '55-60': 0, '60-65': 1001, '65-70': 18067, '70-75': 18067, '75-80': 4057, '80-85': 4057, '85-90': 3, '90-95': 3, '95-100': 1}
{'0-5': 158926, '5-10': 1, '10-15': 2, '15-20': 0, '20-25': 0, '25-30': 0, '30-35': 0, '35-40': 0, '40-45': 0, '45-50': 0, '50-55': 0, '55-60': 0, '60-65': 0, '65-70': 0, '70-75': 0, '75-80': 0, '80-85': 0, '85-90': 0, '90-95': 0, '95-100': 1}


In [19]:
#定义difference weight,取值0~1
def calculate_dw(distribution_1, distribution_2, feature_index):
    count1 = calculate_count(distribution_1, feature_index)
    count2 = calculate_count(distribution_2, feature_index)
    h = 20
    dw = 0
    for i in range(h):
        start = i*5
        end = start + 5
        T1 = distribution_1[str(feature_index)][str(start) + '-' + str(end)]/count1
        T2 = distribution_2[str(feature_index)][str(start) + '-' + str(end)]/count2
        dw += abs(T1 - T2)
    return dw/2
#用于计算分布字典里的实例数
def calculate_count(distribution, feature_index):
    h = 20
    count = 0
    for i in range(h):
        start = i*5
        end = start + 5
        count += distribution[str(feature_index)][str(start) + '-' + str(end)]
    return count

In [20]:
#在属性2下，PortScan与Benign的dw 与 DDoS与Benign的dw 有很大区别
print(calculate_dw(BenignDistributeObj, DDoSDistributeObj, 2))
print(calculate_dw(BenignDistributeObj, PortScanDistributeObj, 2))

0.9963571609103999
0.0038328721707474524


In [44]:
#定义hdw，hybird difference weight
attack_types = 2  #总攻击种类

def calculate_hdw(feature_index):
    
    a = calculate_dw(BenignDistributeObj, DDoSDistributeObj, feature_index) 
    b = calculate_dw(BenignDistributeObj, PortScanDistributeObj, feature_index)
    if(a>b):
        return a
    else:
     return b


In [45]:
#属性2对于构建benign distribution model的差异性权重，也就是模型建立起来的这个属性的重要性考虑度
print(calculate_hdw(2))
print(calculate_hdw(1))
print(calculate_hdw(0))

0.9963571609103999
0.28043034825111096
0.42215406881676354


In [46]:
#计算例子在某种标签上的得分，需要提供对应标签的分布统计字典和字典的统计量
def get_score(example, distributeObj):    
    features_num = 78
    score = 0
    count = calculate_count(distributeObj, 0)  #取哪个特征都一样
    for i in range(78):

        start = int(example[i]/0.05//1)*5   #取200个分组里的分组起始位置
        if(start == 100):     #特殊处理当该属性为1.0时溢出的情况
            start = 95      
        end = start + 5
        dataRange = '{0}-{1}'.format(start, end )
        score += distributeObj[str(i)][dataRange] * calculate_hdw(i) /count / features_num
    return score

In [47]:
print(get_score(Examples["Benign"], BenignDistributeObj))
print(get_score(Examples["DDoS"], BenignDistributeObj))
print(get_score(Examples["PortScan"], BenignDistributeObj))
print("------------")
print(get_score(Examples["Benign"], DDoSDistributeObj))
print(get_score(Examples["DDoS"], DDoSDistributeObj))
print(get_score(Examples["PortScan"], DDoSDistributeObj))
print("------------")
print(get_score(Examples["Benign"], PortScanDistributeObj))
print(get_score(Examples["DDoS"], PortScanDistributeObj))
print(get_score(Examples["PortScan"], PortScanDistributeObj))

0.30014375914518904
0.1768918305007896
0.24936970736841446
------------
0.10284215015540706
0.17995808150822393
0.14571677627872265
------------
0.26621720628199347
0.16812105368904542
0.3222926918713726
